In [1]:
from soundstream import Embedding
import torch,torchaudio

In [2]:
em = Embedding(num_embeddings=512, embedding_dim=512, 
                 start_t=10.0, end_t=0.0, total_epochs=100)


In [3]:
inp = torch.randn((1,512,248))
o = em(inp)

In [4]:
o.shape

torch.Size([1, 512, 248])

In [5]:
em.calc_entropy()

0.012179889716207981

In [6]:
import torchaudio
import torch

In [2]:
model = torch.hub.load("kaiidams/soundstream-pytorch", "soundstream_16khz")
x, sr = torchaudio.load('/ari/users/ibaskaya/projeler/sstream/data/testdata/908_157963_908-157963-0028.in.wav')
x, sr = torchaudio.functional.resample(x, sr, 16000), 16000
with torch.no_grad():
    y = model.encode(x)
    # y = y[:, :, :4]  # if you want to reduce code size.
    z = model.decode(y)
torchaudio.save('/ari/users/ibaskaya/projeler/sstream/data/testdata/outputs/output.wav', z, sr)

Using cache found in /ari/users/ibaskaya/.cache/torch/hub/kaiidams_soundstream-pytorch_main
/ari/users/ibaskaya/.cache/torch/hub/kaiidams_soundstream-pytorch_main/soundstream.py:236: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/ari/users/ibaskaya/.cache/torch/hub/kaiidams_soundstream-pytorch_main/soundstream.py:289: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


In [3]:
z.shape

torch.Size([1, 68706])

In [4]:
aa = model.encoder.forward(x.unsqueeze(0))

In [5]:
bb = torch.transpose(aa, -1, -2)

In [6]:
f,i,l = model.quantizer.forward(bb)

In [7]:
f.shape,i.shape,l.shape

(torch.Size([1, 232, 512]), torch.Size([1, 232, 8]), torch.Size([]))

In [8]:
y.shape

torch.Size([1, 232, 8])

In [9]:
y.shape

torch.Size([1, 232, 8])

In [10]:
x.shape

torch.Size([1, 79280])

In [11]:
z.shape

torch.Size([1, 68706])

In [12]:
dq = model.quantizer.dequantize(i)

In [39]:
dq.shape,f.shape

(torch.Size([1, 232, 512]), torch.Size([1, 232, 512]))

In [40]:
from soundstream import StreamableModel
mmodel = StreamableModel(
        batch_size=32,
        sample_rate=16_000,
        segment_length=32270,
        padding='same',
        dataset='librispeech')
mmodel.load_state_dict(torch.load('/ari/users/ibaskaya/projeler/sstream/lightning_logs/version_10/checkpoints/last.ckpt','cpu')['state_dict'])
_ = mmodel.eval()

/ari/users/ibaskaya/.local/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/tmp/ipykernel_783954/471072808.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` 

In [41]:
x, sr = torchaudio.load('/ari/users/ibaskaya/projeler/sstream/data/testdata/908_157963_908-157963-0028.in.wav')
x, sr = torchaudio.functional.resample(x, sr, 16000), 16000
with torch.no_grad():
    y = mmodel.encode(x.unsqueeze(0))
    # y = y[:, :, :4]  # if you want to reduce code size.
    z = mmodel.decode(y)
torchaudio.save('/ari/users/ibaskaya/projeler/sstream/data/testdata/outputs/output4.wav', z.squeeze(0), sr)

In [44]:
mmodel.quantizer.num_embeddings

1024

In [42]:
y.shape

torch.Size([1, 232, 8])

In [34]:
y.shape,y.dtype, z.shape, z.dtype, x.shape, x.dtype

(torch.Size([1, 232, 2]),
 torch.int64,
 torch.Size([1, 1, 68706]),
 torch.float32,
 torch.Size([1, 79280]),
 torch.float32)

In [36]:
232*8/68706

0.02701365237388292

In [4]:
y.shape

torch.Size([248, 512])

In [5]:
x.shape

torch.Size([1, 79280])

In [6]:
z.shape

torch.Size([1, 78990])

In [7]:
248*512/79280

1.601614530776993

In [8]:
x, sr = torchaudio.load('/ari/users/ibaskaya/projeler/sstream/data/testdata/908_157963_908-157963-0028.in.wav')
x, sr = torchaudio.functional.resample(x, sr, 16000), 16000
a = mmodel.encoder(x)
b = torch.transpose(a, -1, -2)
c, codes, codebook_loss = mmodel.quantizer(b)
d = torch.transpose(c, -1, -2)
e = mmodel.decoder(d)

In [9]:
tt = mmodel.quantizer.dequantize(codes.unsqueeze(0))

In [10]:
tt.shape

torch.Size([1, 248, 512])

In [11]:
d.shape

torch.Size([512, 248])

In [13]:
a.shape,b.shape,c.shape,d.shape,e.shape, codes.shape

(torch.Size([512, 248]),
 torch.Size([248, 512]),
 torch.Size([248, 512]),
 torch.Size([512, 248]),
 torch.Size([1, 78990]),
 torch.Size([248, 1]))

In [14]:
a.dtype,b.dtype,c.dtype,d.dtype,e.dtype, codes.dtype

(torch.float32,
 torch.float32,
 torch.float32,
 torch.float32,
 torch.float32,
 torch.int64)